In [2]:
%pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 32.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk


def download_nltk_resources():
    resources = {
        'tokenizers': ['punkt'],
        'taggers': ['averaged_perceptron_tagger'],
        'corpora': ['stopwords']
    }
    for resource_type, packages in resources.items():
        for package in packages:
            try:
                nltk.data.find(f'{resource_type}/{package}')
            except LookupError:
                nltk.download(package)

In [8]:
import os
import re
import numpy as np
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
##import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')

def extract_features(text):
    """Extracts stylometric features from a single episode's text."""
    # Tokenize words and sentences
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    
    # Filter out punctuation and stopwords
    stop_words = set(stopwords.words('english'))
    words_clean = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
    
    # Part-of-Speech tagging
    pos_tags = pos_tag(words_clean)
    pos_counts = Counter(tag for (word, tag) in pos_tags)
    
    # Feature calculations
    features = {
        # Lexical features
        'word_count': len(words),
        'unique_words': len(set(words)),
        'lexical_diversity': len(set(words)) / len(words) if words else 0,  # Type-Token Ratio (TTR)
        'avg_word_length': np.mean([len(word) for word in words]) if words else 0,
        'hapax_legomena': len([word for word, count in Counter(words).items() if count == 1]),
        
        # Syntactic features
        'avg_sentence_length': np.mean([len(word_tokenize(sent)) for sent in sentences]) if sentences else 0,
        'prop_nouns': pos_counts.get('NN', 0) + pos_counts.get('NNS', 0),
        'prop_verbs': pos_counts.get('VB', 0) + pos_counts.get('VBD', 0) + pos_counts.get('VBG', 0) + pos_counts.get('VBN', 0) + pos_counts.get('VBP', 0) + pos_counts.get('VBZ', 0),
        'prop_adjectives': pos_counts.get('JJ', 0) + pos_counts.get('JJR', 0) + pos_counts.get('JJS', 0),
        'prop_adverbs': pos_counts.get('RB', 0) + pos_counts.get('RBR', 0) + pos_counts.get('RBS', 0),
        'prop_pronouns': pos_counts.get('PRP', 0) + pos_counts.get('PRP$', 0),
        
        # Stylistic features
        'exclamations': text.count('!'),
        'questions': text.count('?'),
        'ellipses': text.count('...'),
    }
    return features

def process_episodes(input_dir):
    """Processes all .txt files in a directory and returns a DataFrame of features."""
    features_list = []
    filenames = []
    
    for filename in sorted(os.listdir(input_dir)):
        if filename.endswith('.txt'):
            with open(os.path.join(input_dir, filename), 'r', encoding='utf-8') as f:
                text = f.read()
                features = extract_features(text)
                features_list.append(features)
                filenames.append(filename)
    
    # Convert to DataFrame
    df = pd.DataFrame(features_list, index=filenames)
    return df

# Example usage
input_directory = "cleaner-text"  # Folder with cleaned .txt files
output_csv = "tp-stylometry.csv"

# Extract features and save to CSV
df_features = process_episodes(input_directory)
df_features.to_csv(output_csv)
print(f"Features extracted and saved to {output_csv}!")

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


Features extracted and saved to tp-stylometry.csv!
